<img src="http://dask.readthedocs.io/en/latest/_images/dask_horizontal.svg"
     align="right"
     width="30%"
     alt="Dask logo\">


# Dask DataFrames

我们通过使用 dask.delayed 在 CSV 文件目录上构建并行数据帧计算完成了第 1 章。 在本节中，我们使用 `dask.dataframe` 自动构建类似的计算，用于表格计算的常见情况。 Dask DataFrame的外观和感觉与 Pandas 数据帧相似，但Dask DataFrame运行在支持`dask.delayed`的相同基础架构上。

在这个笔记本中，我们像以前一样使用相同的航线数据，但是现在我们让`dask.dataframe’`为我们构造计算，而不是写 for循环。函数可以接受`data/nycflights/*`这样的全局字符串,然后在我们所有的数据上建立并行计算。

## 何时使用 `dask.dataframe`

Pandas非常适合存储在内存中的表格数据集。当要分析的数据集大于机器的内存时，Dask 就变得有用了。我们使用的演示数据集大约只有200MB，因此你可以在合理的时间内下载它，但是`dask.dataframe`将扩展到比内存大得多的数据集。

<img src="images/pandas_logo.png" align="right" width="28%">

`dask.dataframe` 模块实现了一个分块的并行 `DataFrame` 对象，它模仿了 Pandas `DataFrame` API 的一个大集合。 一个 Dask `DataFrame` 由许多沿索引分隔的内存中的 Pandas `DataFrames` 组成。 Dask `DataFrame` 上的一个操作会以一种注意潜在并行性和内存限制的方式触发对组成 Pandas `DataFrame` 的许多 Pandas 操作。

**相关文档**

* [DataFrame 文档](https://docs.dask.org/en/latest/dataframe.html)
* [DataFrame 屏幕录像](https://youtu.be/AT2XtFehFSQ)
* [DataFrame API](https://docs.dask.org/en/latest/dataframe-api.html)
* [DataFrame 示例](https://examples.dask.org/dataframe.html)
* [Pandas 文档](https://pandas.pydata.org/pandas-docs/stable/)

**主要知识**

1.  Dask DataFrame对pandas用户来说应该很熟悉
2.  数据流的划分对于有效执行非常重要

## 创建数据

In [ ]:
%run prep.py -d flights

## 设置

In [ ]:
from dask.distributed import Client

client = Client(n_workers=4)

我们创建了人造数据。

In [ ]:
from prep import accounts_csvs
accounts_csvs()

import os
import dask
filename = os.path.join('data', 'accounts.*.csv')
filename

文件名包含一个通配符`*`，因此路径中匹配该通配符的所有文件将被读入同一个的 Dask DataFrame。

In [ ]:
import dask.dataframe as dd
df = dd.read_csv(filename)
df.head()

In [ ]:
# 加载并计算行数
len(df)

这里发生了什么？
- Dask调查了输入路径，发现有3个匹配的文件
- 为每个块智能地创建了一组任务 - 在这种情况下每个原始 CSV 文件对应一个任务
- 每个文件都被加载到一个 Pandas 数据帧中，并应用了 `len()`
- 小计被合并为您最终的总计。

### 真实数据

让我们以几年来在美国的航班为例来尝试一下。这些数据是专门针对纽约市地区三个机场的航班。

In [ ]:
df = dd.read_csv(os.path.join('data', 'nycflights', '*.csv'),
                 parse_dates={'Date': [0, 1, 2]})

请注意，DataFrame对象的描述不包含任何数据 —— Dask 刚刚读取了第一个文件的开头，并推断出了列名和 dtype。

In [ ]:
df

我们可以查看数据的首尾

In [ ]:
df.head()

In [ ]:
df.tail()  # 该操作会失败

### 刚才发生了什么？

与在推断数据类型之前读取整个文件的 `pandas.read_csv` 不同，`dask.dataframe.read_csv` 仅从文件的开头（或第一个文件，如果使用通配）读取样本。然后在读取所有分区时强制执行这些推断的数据类型。

在这种情况下，样本中推断的数据类型不正确。前n行中`CRSElapsedTime`没有值（pandas 将其推断为`float`），后来变成字符串（`object` dtype）。请注意，Dask 会提供有关不匹配的信息性错误消息。发生这种情况时，您有几个选择：

- 直接使用 `dtype` 关键字指定 dtypes。这是推荐的解决方案，因为它最不容易出错（显式比隐式更好），而且性能最高。
- 增加 `sample` 关键字的大小（以字节为单位）
- 使用 `assume_missing` 使 `dask` 假定推断为 `int`（不允许缺失值）的列实际上是浮点数（允许缺失值）。在我们的特殊情况下，这不适用。

在我们的例子中，我们将使用第一个选项并直接指定报错列的 `dtypes`。

In [ ]:
df = dd.read_csv(os.path.join('data', 'nycflights', '*.csv'),
                 parse_dates={'Date': [0, 1, 2]},
                 dtype={'TailNum': str,
                        'CRSElapsedTime': float,
                        'Cancelled': bool})

In [ ]:
df.tail()  # 现在运行成功了

## 使用 `dask.dataframe` 计算

我们计算`DepDelay`列的最大值。 仅使用pandas的情况下，我们将遍历每个文件以找到各个最大值，然后在所有各个最大值上找到最终最大值

```python
maxes = []
for fn in filenames:
    df = pd.read_csv(fn)
    maxes.append(df.DepDelay.max())
    
final_max = max(maxes)
```

我们可以用`dask.delayed`来包装`pd.read_csv`使其并行运行。 无论如何，我们仍然必须考虑循环，中间结果（每个文件一个）和最终减少量（中间最大值的`max`）。这只是围绕真实任务的噪音，pandas用以下代码来解决

```python
df = pd.read_csv(filename, dtype=dtype)
df.DepDelay.max()
```

`dask.dataframe` 让我们编写类似于pandas的代码，该代码可以并行处理大于内存数据集的操作。

In [ ]:
%time df["DepDelay"].max().compute()

这会为我们编写延迟计算，然后运行它。

一些注意事项：

1.与`dask.delayed`一样，我们需要在完成后调用`.compute()`。 到目前为止，一切都是惰性的。
2. Dask 将尽快删除中间结果（如每个文件的完整 Pandas 数据框）。
     - 这让我们可以处理大于内存的数据集
     - 这意味着每次重复计算都必须加载所有数据（再次运行上面的代码，它是否比您预期的更快或更慢？）
    
与`delayed`对象一样，您可以使用`.visualize`方法查看底层任务图：

In [ ]:
# 注意并行
df["DepDelay"].max().visualize()

## 练习

本节中我们进行了一些 `dask.dataframe` 计算. 如果您对pandas很适应，那么这些应该很熟悉. 您将需要思考何时调用 `compute`.

### 1.) 数据集有多少行?

如果您不熟悉pandas，您将如何检查元组列表中有多少条记录？

In [ ]:
# 在这儿输入你的代码

In [ ]:
len(df)

### 2.) 总共有多少非取消（non-canceled）航班？

用pandas的话，您需要使用 [boolean indexing](https://pandas.pydata.org/pandas-docs/stable/indexing.html#boolean-indexing).

In [ ]:
# 在这儿输入你的代码

In [ ]:
len(df[~df.Cancelled])

### 3.) 每个机场总共有多少非取消（non-canceled）航班？

*提示*: 使用 [`df.groupby`](https://pandas.pydata.org/pandas-docs/stable/groupby.html).

In [ ]:
# 在这儿输入你的代码

In [ ]:
df[~df.Cancelled].groupby('Origin').Origin.count().compute()

### 4.) 每个机场的平均起飞延误是多少？

请注意，这与您在之前的笔记本中所做的计算相同（这种方法是更快还是更慢？）

In [ ]:
# 在这儿输入你的代码

In [ ]:
df.groupby("Origin").DepDelay.mean().compute()

### 5.) 一周中哪一天的平均出发延误最严重？

In [ ]:
# 在这儿输入你的代码

In [ ]:
df.groupby("DayOfWeek").DepDelay.mean().compute()

## 共享中间结果

在计算上述所有内容时，我们有时会多次执行相同的操作。 对于大多数操作，`dask.dataframe` 散列参数，允许共享重复计算，并且只计算一次。

例如，让我们计算所有未取消航班的出发延迟的平均值和标准偏差。 由于 dask 操作是惰性的，因此这些值还不是最终结果。 它们只是获得结果所需的配方。

如果我们通过两次计算调用来计算它们，则中间计算不会共享。

In [ ]:
non_cancelled = df[~df.Cancelled]
mean_delay = non_cancelled.DepDelay.mean()
std_delay = non_cancelled.DepDelay.std()

In [ ]:
%%time

mean_delay_res = mean_delay.compute()
std_delay_res = std_delay.compute()

但是让我们尝试将两者都传递给单个 `compute` 调用。

In [ ]:
%%time

mean_delay_res, std_delay_res = dask.compute(mean_delay, std_delay)

使用 `dask.compute` 大约需要 1/2 的时间。 这是因为在调用 `dask.compute` 时合并了两个结果的任务图，允许共享操作只执行一次而不是两次。 特别是，使用 `dask.compute` 只执行以下一次：

- 调用 `read_csv`
- 过滤器（`df[~df.Cancelled]`）
- 一些必要的归约（`sum`，`count`）

要查看多个结果之间的合并任务图是什么样的（以及共享的内容），您可以使用 `dask.visualize` 函数（我们可能希望使用 `filename='graph.pdf'` 将图形保存到磁盘，以便我们可以更轻松地放大）：

In [ ]:
dask.visualize(mean_delay, std_delay)

## 这与 Pandas 相比如何？

Pandas 比 dask.dataframe 更成熟、功能更齐全。如果您的数据适合内存，那么您应该使用 Pandas。当您对不适合内存的数据集进行操作时，`dask.dataframe` 模块为您提供了有限的 `pandas` 体验。

在本教程中，我们提供了一个由几个 CSV 文件组成的小数据集。这个数据集在磁盘上有 45MB，在内存中扩展到大约 400MB。该数据集足够小，您通常可以使用 Pandas。

我们选择了这个尺寸，以便练习快速完成。 Dask.dataframe 只有对比这大得多的问题才真正变得有意义，此时，Pandas 打破了可怕的

    MemoryError:  ...
    
此外，分布式调度器允许跨集群执行相同的数据帧表达式。为了实现海量“大数据”处理，可以执行数据摄取功能，例如`read_csv`，其中数据保存在每个工作节点（例如亚马逊的 S3）都可以访问的存储中，并且因为大多数操作从仅选择一些列开始，转换和过滤数据，只需要在机器之间通信相对少量的数据。

Dask.dataframe 操作在内部使用 `pandas` 操作。除了以下两种情况外，它们通常以大致相同的速度运行：

1. Dask 引入了一些开销，每个任务大约 1 毫秒。这通常可以忽略不计。
2. 当 Pandas释放GIL 时，`dask.dataframe` 可以在一个进程内并行调用多个 Pandas 操作，提高速度与内核数量成正比。对于不释放 GIL 的操作，需要多个进程才能获得相同的加速。

## Dask DataFrame 数据模型

在大多数情况下，Dask DataFrame 感觉就像一个 Pandas DataFrame。
到目前为止，我们看到的最大区别是 Dask 操作是惰性的； 他们建立了一个任务图而不是立即执行（更多细节见 [调度器](05_distributed.ipynb)）。
这让 Dask 可以在内核外并行执行操作。

在[Dask Arrays](03_array.ipynb)中, 我们看到一个 `dask.array` 由许多 NumPy 数组组成，沿着一个或多个维度分块。
在`dask.dataframe`中也是相似的: Dask DataFrame 由许多 Pandas DataFrame 组成。 对于`dask.dataframe`，分块仅沿索引发生。

<img src="http://dask.pydata.org/en/latest/_images/dask-dataframe.svg" width="30%">

我们称每个块为 *partition*，上限/下限为 *division*。
Dask *可以* 存储有关division的信息。 目前，当您编写自定义函数以应用于 Dask DataFrame 时会出现分区。

## 将 `CRSDepTime` 转化为时间戳

该数据集将时间戳存储为`HHMM`，在`read_csv`中作为整数读入：

In [ ]:
crs_dep_time = df.CRSDepTime.head(10)
crs_dep_time

要将这些转换为预定出发时间的时间戳，我们需要将这些整数转换为 `pd.Timedelta` 对象，然后将它们与 `Date` 列组合。

在 Pandas 中，我们会使用 `pd.to_timedelta` 函数和一些算术来做到这一点：

In [ ]:
import pandas as pd

# 获取前 10 个日期来补充我们的 `crs_dep_time`
date = df["Date"].head(10)

# 以整数形式获取小时数，转换为时间增量(timedelta)
hours = crs_dep_time // 100
hours_timedelta = pd.to_timedelta(hours, unit='h')

# 以整数形式获取分钟数，转换为时间增量(timedelta)
minutes = crs_dep_time % 100
minutes_timedelta = pd.to_timedelta(minutes, unit='m')

# 应用 timedeltas 以按出发时间偏移日期
departure_timestamp = date + hours_timedelta + minutes_timedelta
departure_timestamp

### 自定义代码和Dask Dataframe

我们可以将 `pd.to_timedelta` 换成 `dd.to_timedelta`，并对整个 dask DataFrame 执行相同的操作。 但是假设 Dask 还没有实现适用于 Dask DataFrame 的 `dd.to_timedelta`。 那你会怎么做？

`dask.dataframe` 提供了一些方法来更容易地将自定义函数应用于 Dask DataFrames：

- [`map_partitions`](http://dask.pydata.org/en/latest/dataframe-api.html#dask.dataframe.DataFrame.map_partitions)
- [`map_overlap`](http://dask.pydata.org/en/latest/dataframe-api.html#dask.dataframe.DataFrame.map_overlap)
- [`reduction`](http://dask.pydata.org/en/latest/dataframe-api.html#dask.dataframe.DataFrame.reduction)

在这里，我们将只讨论 `map_partitions`，我们可以使用它来自己实现 `to_timedelta`：

In [ ]:
# 查看`map_partitions`的文档

help(df["CRSDepTime"].map_partitions)

基本思想是将一个对 DataFrame 进行操作的函数应用于每个分区。
在这种情况下，我们将应用 `pd.to_timedelta`。

In [ ]:
hours = df["CRSDepTime"] // 100
# hours_timedelta = pd.to_timedelta(hours, unit='h')
hours_timedelta = hours.map_partitions(pd.to_timedelta, unit='h')

minutes = df["CRSDepTime"] % 100
# minutes_timedelta = pd.to_timedelta(minutes, unit='m')
minutes_timedelta = minutes.map_partitions(pd.to_timedelta, unit='m')

departure_timestamp = df["Date"] + hours_timedelta + minutes_timedelta

In [ ]:
departure_timestamp

In [ ]:
departure_timestamp.head()

### 练习：重写上面的代码以使用对 `map_partitions` 的单个调用

这将比两个单独的调用稍微更有效，因为它减少了图中的任务数量。

In [ ]:
def compute_departure_timestamp(df):
    pass  # 目标：完成它

In [ ]:
departure_timestamp = df.map_partitions(compute_departure_timestamp)

departure_timestamp.head()

In [ ]:
def compute_departure_timestamp(df):
    hours = df.CRSDepTime // 100
    hours_timedelta = pd.to_timedelta(hours, unit='h')

    minutes = df.CRSDepTime % 100
    minutes_timedelta = pd.to_timedelta(minutes, unit='m')

    return df.Date + hours_timedelta + minutes_timedelta

departure_timestamp = df.map_partitions(compute_departure_timestamp)
departure_timestamp.head()

## 局限性

### 什么不起作用？

Dask.dataframe 只涵盖了 Pandas API 中较小，但广泛使用的部分。
这种限制有两个原因：

1. Pandas API真的*很多*
2. 一些操作真的很难并行执行（例如排序）

此外，一些重要的操作，如 ``set_index`` 可以用dask.dataframe实现，但与 Pandas 相比，速度会较慢，因为它们包括大量的数据混洗，并且可能会写出到磁盘。

## 了解更多


* [DataFrame 文档](https://docs.dask.org/en/latest/dataframe.html)
* [DataFrame 屏幕录像](https://youtu.be/AT2XtFehFSQ)
* [DataFrame API](https://docs.dask.org/en/latest/dataframe-api.html)
* [DataFrame 示例](https://examples.dask.org/dataframe.html)
* [Pandas 文档](https://pandas.pydata.org/pandas-docs/stable/)

In [ ]:
client.shutdown()